In [ ]:
# Ingredients to be used in 10 recipes

# ["bacon", "eggs", "buns", "tomatoes", "mayonnaise"]
# ["bay scallops", "butter", "onions", "parsley", "lemons", "bread crumbs"]
# ["rice", "chicken", "cream of mushroom soup", "cream of chicken soup", "onion soup", "water"]
# ["coriander", "allspice", "cumin", "ground ginger", "turmeric", "cinnamon", "cayenne", "pepper", "salt", "lemon juice", "olive oil", "garlic", "chicken", "pita", "red onion", "parsley", "shredded red cabbage", "tahini"]
# ["sour cream", "heavy cream", "baking powder", "kosher salt", "sugar", "ground black pepper", "flour", "butter", "salt", "cutter"]
# ["hamburger", "egg", "bread crumbs", "onion", "milk", "salt", "pepper", "bacon", "catsup"]
# ["stew meat", "cabbage", "onion", "green bell peppers", "oil", "tomatoes", "garlic", "soy sauce"]
# ["chicken broth", "water", "chicken", "onion", "vegetables", "garlic", "salt", "fresh cilantro", "potatoes", "vegetable oil"]
# ["pork sparerib", "black bean sauce", "Chinese rice wine", "cornstarch", "ginger", "garlic", "ground black pepper", "cooking oil", "sesame oil", "sugar", "red chili pepper"]
# ["shell pasta", "red pepper", "green onion", "seafood", "peas", "olive oil", "sesame oil", "rice vinegar", "lemon", "mayonnaise"]

In [ ]:
# Importing the required libraries & modules
from langchain_community.document_loaders.csv_loader import CSVLoader


In [2]:
# import pyodbc
from langchain_community.utilities import SQLDatabase

server = 'rag_application_server'
database_name = 'personalized_recipe_generator'
user = 'root'
password = 'root'
# connection_string = f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database_name};UID={user};PWD={password}'
# conn = pyodbc.connect(connection_string)

host="localhost"
port="3306"

In [2]:
import pandas as pd

import os
current_path = os.getcwd()
print(current_path)

os.chdir('../dataset/')
path_base_dataset = os.getcwd()
print(path_base_dataset)

os.chdir('../exploratory_data_analysis/')
path_base_notebook = os.getcwd()
print(path_base_notebook)

NUM_DATA_SAMPLES = 50000
file_name = '/sampled_dataset_' + str(NUM_DATA_SAMPLES) + '.csv'
sampled_data_file_path = path_base_dataset + file_name

df = pd.read_csv(sampled_data_file_path)
df.head()

c:\Users\DCW100940297\Documents\github\Personalized_Recipe_Generator\exploratory_data_analysis
c:\Users\DCW100940297\Documents\github\Personalized_Recipe_Generator\dataset
c:\Users\DCW100940297\Documents\github\Personalized_Recipe_Generator\exploratory_data_analysis


,Unnamed: 0,ID,name,ingredients,procedure,link,source,NER
0,1254789,1254789,"Bacon, Lettuce, Tomato And Fried Egg Sandwiches","[""4 slices bacon, halved"", ""4 None eggs"", ""4 N...","[""Lightly coat a nonstick frying pan and place...",recipes-plus.com/api/v2.0/recipes/37658,Gathered,"[""bacon"", ""eggs"", ""buns"", ""tomatoes"", ""mayonna..."
1,2162678,2162678,Sauteed Bay Scallops Recipe,"[""2 pound bay scallops"", ""1/4 pound butter, me...","[""Saute/fry onions and parsley in butter."", ""R...",cookeatshare.com/recipes/sauteed-bay-scallops-...,Recipes1M,"[""bay scallops"", ""butter"", ""onions"", ""parsley""..."
2,832049,832049,No Fuss No Mess Chicken And Rice,"[""1 c. uncooked rice"", ""1 chicken, cut up"", ""1...","[""Place rice in oblong pan."", ""Place chicken o...",www.cookbooks.com/Recipe-Details.aspx?id=334186,Gathered,"[""rice"", ""chicken"", ""cream of mushroom soup"", ..."
3,1396186,1396186,Baked Chicken Shawarma,"[""2 teaspoons coriander"", ""1 teaspoon allspice...","[""In a large bowl combine coriander, allspice,...",food52.com/recipes/41562-baked-chicken-shawarma,Gathered,"[""coriander"", ""allspice"", ""cumin"", ""ground gin..."
4,1322877,1322877,Salt-And-Pepper Biscuits,"[""3/4 cup sour cream"", ""4 tablespoons heavy cr...","[""Preheat oven to 425\u00b0F. Whisk sour cream...",www.epicurious.com/recipes/food/views/salt-and...,Gathered,"[""sour cream"", ""heavy cream"", ""baking powder"",..."


In [6]:
from langchain_community.utilities import SQLDatabase
from langchain_core.messages import AIMessage, HumanMessage

def init_database(user: str, password: str, host: str, port: str, database: str) -> SQLDatabase:
  db_uri = f"mysql+mysqlconnector://{user}:{password}@{host}:{port}/{database}"
  return SQLDatabase.from_uri(db_uri)

db = init_database(
  user="root",
  password="root",
  host="localhost",
  port="3306",
  database=database_name
  )

DatabaseError: (mysql.connector.errors.DatabaseError) 2003 (HY000): Can't connect to MySQL server on 'localhost:3306' (10061)
(Background on this error at: https://sqlalche.me/e/20/4xp6)

In [ ]:
conn = f"mysql+mysqlconnector://{user}:{password}@{host}:{port}/{database_name}"
df.to_sql(name='personalized_recipe_generator', con=conn, if_exists='replace', index=False)

In [ ]:
print(db.dialect)
print(db.get_usable_table_names())

In [2]:
from langchain_community.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(sampled_data_file_path)
docs = loader.load()

In [8]:
docs[0].page_content

': 1254789\nID: 1254789\nname: Bacon, Lettuce, Tomato And Fried Egg Sandwiches\ningredients: ["4 slices bacon, halved", "4 None eggs", "4 None hamburger buns, toasted", "4 None lettuce leaves", "2 None tomatoes, sliced", "None None mayonnaise or barbecue sauce"]\nprocedure: ["Lightly coat a nonstick frying pan and place over high heat. Fry bacon for 3 mins, turning, or until browned. Transfer to a plate and keep warm.", "Reduce heat to medium-low. Fry eggs until cooked to your liking.", "Top bottom burger buns with lettuce, bacon, tomato and egg. Add sauce of your choice then top with top buns. Serve."]\nlink: recipes-plus.com/api/v2.0/recipes/37658\nsource: Gathered\nNER: ["bacon", "eggs", "buns", "tomatoes", "mayonnaise"]'

In [10]:
df.head()

,Unnamed: 0,ID,name,ingredients,procedure,link,source,NER
0,1254789,1254789,"Bacon, Lettuce, Tomato And Fried Egg Sandwiches","[""4 slices bacon, halved"", ""4 None eggs"", ""4 N...","[""Lightly coat a nonstick frying pan and place...",recipes-plus.com/api/v2.0/recipes/37658,Gathered,"[""bacon"", ""eggs"", ""buns"", ""tomatoes"", ""mayonna..."
1,2162678,2162678,Sauteed Bay Scallops Recipe,"[""2 pound bay scallops"", ""1/4 pound butter, me...","[""Saute/fry onions and parsley in butter."", ""R...",cookeatshare.com/recipes/sauteed-bay-scallops-...,Recipes1M,"[""bay scallops"", ""butter"", ""onions"", ""parsley""..."
2,832049,832049,No Fuss No Mess Chicken And Rice,"[""1 c. uncooked rice"", ""1 chicken, cut up"", ""1...","[""Place rice in oblong pan."", ""Place chicken o...",www.cookbooks.com/Recipe-Details.aspx?id=334186,Gathered,"[""rice"", ""chicken"", ""cream of mushroom soup"", ..."
3,1396186,1396186,Baked Chicken Shawarma,"[""2 teaspoons coriander"", ""1 teaspoon allspice...","[""In a large bowl combine coriander, allspice,...",food52.com/recipes/41562-baked-chicken-shawarma,Gathered,"[""coriander"", ""allspice"", ""cumin"", ""ground gin..."
4,1322877,1322877,Salt-And-Pepper Biscuits,"[""3/4 cup sour cream"", ""4 tablespoons heavy cr...","[""Preheat oven to 425\u00b0F. Whisk sour cream...",www.epicurious.com/recipes/food/views/salt-and...,Gathered,"[""sour cream"", ""heavy cream"", ""baking powder"",..."


In [4]:
from langchain_community.document_loaders import DataFrameLoader

loader = DataFrameLoader(df, page_content_column="NER")
docs = loader.load()

In [5]:
docs[0].metadata

{'Unnamed: 0': 1254789,
 'ID': 1254789,
 'name': 'Bacon, Lettuce, Tomato And Fried Egg Sandwiches',
 'ingredients': '["4 slices bacon, halved", "4 None eggs", "4 None hamburger buns, toasted", "4 None lettuce leaves", "2 None tomatoes, sliced", "None None mayonnaise or barbecue sauce"]',
 'procedure': '["Lightly coat a nonstick frying pan and place over high heat. Fry bacon for 3 mins, turning, or until browned. Transfer to a plate and keep warm.", "Reduce heat to medium-low. Fry eggs until cooked to your liking.", "Top bottom burger buns with lettuce, bacon, tomato and egg. Add sauce of your choice then top with top buns. Serve."]',
 'link': 'recipes-plus.com/api/v2.0/recipes/37658',
 'source': 'Gathered'}

In [6]:
docs[0].page_content

'["bacon", "eggs", "buns", "tomatoes", "mayonnaise"]'

In [7]:
from langchain.indexes import VectorstoreIndexCreator

vectorstore_index = VectorstoreIndexCreator().from_loaders([docs])

c:\Users\DCW100940297\AppData\Local\anaconda3\envs\MLOPs_LLM_Zoomcamp\Lib\site-packages\langchain\indexes\vectorstore.py:128: UserWarning: Using InMemoryVectorStore as the default vectorstore.This memory store won't persist data. You should explicitlyspecify a vectorstore when using VectorstoreIndexCreator
  warnings.warn(


ValidationError: 1 validation error for VectorstoreIndexCreator
embedding
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/missing

In [8]:
import duckdb

duckdb.read_csv(sampled_data_file_path)

┌─────────┬─────────┬─────────────────────────────────────────────────┬──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [12]:
conn = duckdb.connect('recipe_dataset.db')
conn.execute(
f" CREATE TABLE recipe_tbl AS SELECT * FROM read_csv('{sampled_data_file_path}');"
)

result = conn.execute(" SELECT * FROM recipe_tbl").fetchall()

conn.close


In [13]:
result

[(1254789,
  1254789,
  'Bacon, Lettuce, Tomato And Fried Egg Sandwiches',
  '["4 slices bacon, halved", "4 None eggs", "4 None hamburger buns, toasted", "4 None lettuce leaves", "2 None tomatoes, sliced", "None None mayonnaise or barbecue sauce"]',
  '["Lightly coat a nonstick frying pan and place over high heat. Fry bacon for 3 mins, turning, or until browned. Transfer to a plate and keep warm.", "Reduce heat to medium-low. Fry eggs until cooked to your liking.", "Top bottom burger buns with lettuce, bacon, tomato and egg. Add sauce of your choice then top with top buns. Serve."]',
  'recipes-plus.com/api/v2.0/recipes/37658',
  'Gathered',
  '["bacon", "eggs", "buns", "tomatoes", "mayonnaise"]'),
 (2162678,
  2162678,
  'Sauteed Bay Scallops Recipe',
  '["2 pound bay scallops", "1/4 pound butter, melted", "1 c. finely minced onions", "1/2 bunch parsley, finely minced", "Juice of 2 lemons", "1 c. bread crumbs"]',
  '["Saute/fry onions and parsley in butter.", "Remove from heat.", "Pat

In [18]:
conn_test = duckdb.connect('recipe_dataset.db')

conn_test.query('describe recipe_tbl;')


┌─────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│ column_name │ column_type │  null   │   key   │ default │  extra  │
│   varchar   │   varchar   │ varchar │ varchar │ varchar │ varchar │
├─────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ column0     │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ ID          │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ name        │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ ingredients │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ procedure   │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ link        │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ source      │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ NER         │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
└─────────────┴─────────────┴─────────┴─────────┴─────────┴─────────┘

In [19]:
conn_test.close()

In [20]:
from langchain_community.utilities import SQLDatabase

db_uri = 'duckdb:///recipe_dataset.db'

db = SQLDatabase.from_uri(db_uri)

OperationalError: (duckdb.duckdb.ConnectionException) Connection Error: Can't open a connection to same database file with a different configuration than existing connections
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [ ]:
print(db.get_usable_table_names())